<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NATURAL-ISOLATES---NOISEQ/CQN/DESEQ2" data-toc-modified-id="NATURAL-ISOLATES---NOISEQ/CQN/DESEQ2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NATURAL ISOLATES - NOISEQ/CQN/DESEQ2</a></span><ul class="toc-item"><li><span><a href="#KEYS" data-toc-modified-id="KEYS-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>KEYS</a></span></li></ul></li><li><span><a href="#PREPPING-READ-DATA" data-toc-modified-id="PREPPING-READ-DATA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PREPPING READ DATA</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#UNNORMALIZED-GENE-COUNTS" data-toc-modified-id="UNNORMALIZED-GENE-COUNTS-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>UNNORMALIZED GENE COUNTS</a></span></li><li><span><a href="#GATHERING-FRAGMENT-LENGTHS-FROM-GFF-FILE" data-toc-modified-id="GATHERING-FRAGMENT-LENGTHS-FROM-GFF-FILE-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>GATHERING FRAGMENT LENGTHS FROM GFF FILE</a></span></li></ul></li><li><span><a href="#INSTALLING-PACKAGES-AND-LOADING-LIBRARIES" data-toc-modified-id="INSTALLING-PACKAGES-AND-LOADING-LIBRARIES-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>INSTALLING PACKAGES AND LOADING LIBRARIES</a></span></li><li><span><a href="#NOISEQ" data-toc-modified-id="NOISEQ-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NOISEQ</a></span><ul class="toc-item"><li><span><a href="#PRE-FILTERING-THROUGH-NOISEQ" data-toc-modified-id="PRE-FILTERING-THROUGH-NOISEQ-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>PRE-FILTERING THROUGH NOISEQ</a></span></li><li><span><a href="#FILTERED-GC-CONTENT" data-toc-modified-id="FILTERED-GC-CONTENT-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>FILTERED GC CONTENT</a></span></li><li><span><a href="#DESEQ-NORMALIZING-NOISEQ-FILTERED-DATA" data-toc-modified-id="DESEQ-NORMALIZING-NOISEQ-FILTERED-DATA-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>DESEQ NORMALIZING NOISEQ FILTERED DATA</a></span></li><li><span><a href="#ADDING-GC-CONTENT-TO-DESEQ2-NORMALIZED-NOISEQ-FILTERED-GENE-COUNTS" data-toc-modified-id="ADDING-GC-CONTENT-TO-DESEQ2-NORMALIZED-NOISEQ-FILTERED-GENE-COUNTS-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>ADDING GC CONTENT TO DESEQ2 NORMALIZED NOISEQ FILTERED GENE COUNTS</a></span></li><li><span><a href="#CREATING-NOISEQ-DATA-OBJECT" data-toc-modified-id="CREATING-NOISEQ-DATA-OBJECT-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>CREATING NOISEQ DATA OBJECT</a></span></li></ul></li><li><span><a href="#CONDITIONAL-QUANTILE-NORMALIZATION" data-toc-modified-id="CONDITIONAL-QUANTILE-NORMALIZATION-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CONDITIONAL QUANTILE NORMALIZATION</a></span></li><li><span><a href="#DESEQ2---DIFFERENTIAL-EXPRESSION-ANALYSIS" data-toc-modified-id="DESEQ2---DIFFERENTIAL-EXPRESSION-ANALYSIS-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DESEQ2 - DIFFERENTIAL EXPRESSION ANALYSIS</a></span></li><li><span><a href="#GB-STRAINS---EXPRESSION-VARIATION-PER-INCREASING-SAMPLE-SIZE" data-toc-modified-id="GB-STRAINS---EXPRESSION-VARIATION-PER-INCREASING-SAMPLE-SIZE-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>GB STRAINS - EXPRESSION VARIATION PER INCREASING SAMPLE SIZE</a></span></li></ul></li></ul></div>

# NATURAL ISOLATES - NOISEQ/CQN/DESEQ2

## KEYS

In [ ]:
import json
import pandas as pd
import numpy as np
from numpy import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from matplotlib.gridspec import GridSpec
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import fligner
from scipy.stats import skewtest
stats.junk = lambda chisq, df: stats.chi2.sf(chisq, df)
import csv
import gffpandas.gffpandas as gffpd
import sklearn
from sklearn.metrics import r2_score
import os
import statistics
%load_ext rpy2.ipython

#### DATAFRAME RECORDING THE GENERATION TIME PER SAMPLE ####
dic_gen = {'CC2344-L1': 912.356113, 'CC2344-L10': 917.129696, 'CC2344-L11': 889.5859554, 'CC2344-L12': 950.0552184, 'CC2344-L13': 961.4186064,
           'CC2344-L14': 931.447801, 'CC2344-L15': 946.6643063, 'CC2344-L2': 923.1078072, 'CC2344-L3': 1000.469526, 'CC2344-L4': 808.9505794,
           'CC2344-L5': 957.6380465, 'CC2344-L6': 970.6307256, 'CC2344-L7': 990.9451516, 'CC2344-L8': 1009.966123, 'CC2344-L9': 901.0619061, 
           'CC2931-L1': 1050.109001, 'CC2931-L10': 1097.978141, 'CC2931-L11': 1021.13559, 'CC2931-L13': 1041.362593, 
           'CC2931-L14': 1016.111493, 'CC2931-L15': 1052.540951, 'CC2931-L2': 1056.765369, 'CC2931-L3': 1000.399127, 'CC2931-L4': 1011.411706,
           'CC2931-L5': 993.8603657, 'CC2931-L6': 1083.095655, 'CC2931-L7': 1067.34507, 'CC2931-L9': 1079.236285}
generations = pd.Series(dic_gen)

#### DATAFRAME RECORDING THE NUMBER OF MUTATIONS PER SAMPLE ####
dic_mut = {'CC2344-L1': 396, 'CC2344-L10': 59, 'CC2344-L11': 46, 'CC2344-L12': 74, 'CC2344-L13': 49, 'CC2344-L14': 46, 'CC2344-L15': 53, 
           'CC2344-L2': 80, 'CC2344-L3': 63, 'CC2344-L4': 24, 'CC2344-L5': 68, 'CC2344-L6': 38, 'CC2344-L7': 45, 'CC2344-L8': 75, 'CC2344-L9': 27, 
           'CC2931-L1': 89, 'CC2931-L10': 87, 'CC2931-L11': 85, 'CC2931-L13': 97, 'CC2931-L14': 79, 'CC2931-L15': 141, 'CC2931-L2': 123,
           'CC2931-L3': 52, 'CC2931-L4': 100, 'CC2931-L5': 335, 'CC2931-L6': 84, 'CC2931-L7': 72, 'CC2931-L9': 113}
mutations = pd.Series(dic_mut)

# PREPPING READ DATA

### UNNORMALIZED GENE COUNTS 

In [ ]:
# ## NAMES OF GENE COUNT FILES ##
# keyword = 'genes_count'
# label = []
# for fname in os.listdir('/research/projects/chlamydomonas/MAexpression/data/gene_count/'):
#     if keyword in fname:
#         fname = fname.replace('_genes_count','')
#         if 'GB' in fname:
#             label.append(fname)

# ## GB NATURAL STRAINS - UNNORMALIZED GENE COUNTS ##
# gene_count = pd.DataFrame()
# for i in label:
#     section = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/gene_count/' + i + '_genes_count', delimiter = '\t', names = ['name', i])
#     gene_count = pd.concat([gene_count, section], axis = 1)
    
# gene_count = gene_count.loc[:,~gene_count.columns.duplicated()]
# gene_count = gene_count.drop(gene_count.index.values[-5:])
# gene_count = gene_count.set_index('name')

# ## EXPORTING
# gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/unnormalized_GB_raw.txt', sep = '\t', index = True, header = True)

### GATHERING FRAGMENT LENGTHS FROM GFF FILE

In [ ]:
# gff = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v6_genome_plus_anno/CC4532.v1_1.gene_exons.gff3', skiprows=3, delimiter = '\t')
# gff.columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
# gff[['attributes1', 'attributes2', 'attributes3', 'attributes4']] = gff['attributes'].str.split(';', n = 3, expand = True)
# gff_exon = gff.loc[gff['type'].isin(['mRNA', 'exon'])]
# gff_exon[['attributes1', 'attributes2', 'attributes4']] = gff_exon[['attributes1', 'attributes2', 'attributes4']].replace({'Parent=':'', 'ID=':''}, regex = True)
# gff_exon_pos = gff_exon.loc[gff_exon['strand'] == '+']
# gff_exon_neg = gff_exon.loc[gff_exon['strand'] == '-']
# mRNA = gff_exon.loc[gff_exon['type'] == 'mRNA'].reset_index()
# mRNA = mRNA.drop('index', axis = 1)
# mRNA['attributes'] = mRNA['attributes'].str.split(';')
# mRNA['parent'] = 'nan'
# for i in list(mRNA.index.values):
#     for a in mRNA.at[i,'attributes']:
#         if 'Parent=' in a:
#             mRNA.at[i, 'parent'] = a

# ## EXTRACTING THE TOTAL SUM OF EXONS PER GENE
# mRNA['total_exon_length'] = 'nan'
# mRNA.rename(columns={'attributes1': 'ID'}, inplace = True)
# mRNA_ID = mRNA['ID'].values.tolist()
# gff_exon = gff_exon.loc[gff_exon['type'] == 'exon']

# for i in range(len(mRNA_ID)):
#     gene_length = []
#     section = gff_exon.loc[gff_exon['attributes2'] == mRNA_ID[i]]
#     for a in list(section.index.values):
#         gene_length.append(section.at[a,'end'] - section.at[a, 'start'])
#     mRNA.at[i, 'total_exon_length'] = sum(gene_length)
# mRNA_1 = mRNA.drop_duplicates(subset=['parent'], keep='last')

# fragment_length = pd.Series(data = mRNA_1['total_exon_length'].values.tolist(), index = list(parent), name = 'total_exon_length')

# ## EXPORT DATA
# fragment_length.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', sep = '\t', index = True, index_label= 'index', header = True)

## INSTALLING PACKAGES AND LOADING LIBRARIES

In [ ]:
%%R
## TIDYVERSE
library(tidyverse)

## DESEQ2
library("BiocParallel")
register(MulticoreParam(4))
library("DESeq2")

## CQN
library(cqn)
library(scales)

## NOISEQ
library(NOISeq)

## NOISEQ

### PRE-FILTERING THROUGH NOISEQ

In [ ]:
%%R

#######################################
### NOISEQ PRE-FILTERING FOR GB ###
#######################################
GB <- data.frame(read.table('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//unnormalized_GB_raw.txt', sep = '\t', header = TRUE, row.names = 'name', stringsAsFactors = FALSE))
factor <- colnames(GB)[1:length(GB)]
factor[grep('GB', factor)] <- 'treatment'
factor <- matrix(factor, nrow = length(GB), ncol = 1)
filtered_GB <- filtered.data(GB, factor, norm=FALSE, method=2, p.adj='BH')


## EXPORTING PREFILTERED TEXTFILE
rite.table(as.data.frame(filtered_GB), 'raw_counts//noiseq_filtered_unnormalized_GB.txt', sep="\t", quote = F)

### FILTERED GC CONTENT

In [ ]:
## OPENING GC LENGTH AND FRAGMENT LENGTH
gc_content = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v6_genome_plus_anno/GC_lengths.tsv', delimiter = '\t')
fragment_length = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/fpkm/fragment_length.csv', delimiter = '\t')
fragment_length = fragment_length.set_index('Unnamed: 0')

## OPENING NOISEQ FILTERED GENE COUNTS
GB_gene_count = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/noiseq_filtered_unnormalized_GB.txt', delimiter = '\t')
B_gene_count = GB_gene_count.join([fragment_length, gc_content['GC']])

## EXPORT ##
GB_gene_count.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_filtered_noiseq_wt_GC', sep = '\t', index = True, header = True)

### DESEQ NORMALIZING NOISEQ FILTERED DATA

In [ ]:
%%R

################
#### GB ####
################
GB <- data.frame(read.csv('/research//projects//chlamydomonas//MAexpression//analysis//raw_counts//GB_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, row.names = 'X', stringsAsFactors = FALSE))
condition <- colnames(GB)[1:(length(GB)-2)]
counts <- GB[condition]
condition_1 <- condition
condition[grep('GB', condition)] <- 'treatment'

coldata <- data.frame(condition, condition_1, row.names = 'condition_1')

dds <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ condition)
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
dds <- estimateSizeFactors(dds)
counts <- counts(dds, normalized=TRUE)
write.table(counts, paste(directory, "GB_noiseq_filtered_deseq_normalized", ".txt", sep=""), sep="\t", quote = F)

### ADDING GC CONTENT TO DESEQ2 NORMALIZED NOISEQ FILTERED GENE COUNTS

In [ ]:
# ## OPENING DESEQ2 NORMALIZED NOISEQ FILTERED GENE COUNTS
GB_normalized_GC = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_noiseq_filtered_deseq_normalized.txt', delimiter = '\t')
GB_normalized_GC = GB_normalized_GC.join([fragment_length, gc_content['GC']])

# ## EXPORT ##
GB_normalized_GC.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', index = True, header = True)

### CREATING NOISEQ DATA OBJECT

In [ ]:
GB_deseq_normalized <- data.frame(read.table('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_deseq_normalized_noiseq_filtered_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE))
GB_length <- data.frame(GB_deseq_normalized$total_exon_length)
GB_GC <- data.frame(GB_deseq_normalized$GC)

## PROVIDING INDEXES
row.names(GB_length) <- GB_deseq_normalized$X
row.names(GB_GC) <- GB_deseq_normalized$X
row.names(GB_deseq_normalized) <- GB_deseq_normalized$X

## CREATING FACTORS AND COUNTS
GB_counts <- GB_deseq_normalized[colnames(GB_deseq_normalized)[2:(length(GB_deseq_normalized)-2)]]
GB_factor <- colnames(GB_counts)
GB_columns <- GB_factor
colnames(GB_counts) <- GB_columns

label <- c('GB117', 'GB119', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123')
for (i in label) {GB_factor[grep(i, label)] <- i}

GB_factor <- data.frame(GB_factor)

# ## CREATING READDATA FILES
data_GB <- readData(data = GB_counts, length = GB_length, gc = GB_GC, factors = GB_factor)

## CONDITIONAL QUANTILE NORMALIZATION

In [ ]:
####################################
#### GB - CQN NORMALIZATION ####
####################################
directory <- "/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/"
GB_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
counts <- GB_noiseq_filtered[c(1:24)]
cqn_res <- cqn(counts = counts, x = GB_noiseq_filtered$GC, lengths = GB_noiseq_filtered$total_exon_length)

#### CREATING FACTORS AND COLDATA ####
condition <- c('GB117', 'GB119', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123')
factor <- colnames(counts)
columns <- factor
for (i in condition) {factor[grep((i), factor)] <- i}
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file = "cqn/GB_gc_plot_before.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(3, 10))
dev.off()

jpeg(file = "cqn/GB_length_plot_before.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(4, 8))
dev.off()

#### NORMALIZED EXPRESSION VALUES ####
cqnnorm <- cqn_res$y + cqn_res$offset
cqnOffset <- cqn_res$glm.offset
cqnNormFactors <- exp(cqnOffset)
cqnNormFactors[is.na(cqnNormFactors)] <- 1 ## replacing missing values with 1
normFactors_sameScale <- cqnNormFactors/exp(rowMeans(log(cqnNormFactors)))

#### CREATE DATASET USING NORMFACTORS_SAMESCALE ####
GB_dds1 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(GB_dds1) <- normFactors_sameScale
counts_nobias_sameScale = counts(GB_dds1, normalized = TRUE)
write.table(counts_nobias_sameScale, paste(directory, "GB_noiseq_filtered_cqn_normalized_samescale", ".txt", sep=""), sep="\t", quote = F)

#### CREATE DATASET USING NORMFACTORS ####
GB_dds2 <- DESeqDataSetFromMatrix(countData = counts, colData = coldata, design = ~ factor)
normalizationFactors(GB_dds2) = cqnNormFactors
counts_nobias = counts(GB_dds2, normalized = TRUE)
write.table(counts_nobias, paste(directory, "GB_noiseq_filtered_cqn_normalized", ".txt", sep=""), sep="\t", quote = F)

In [ ]:
################################################
### GB - RECHECKING CQN NORMALIZED DATA ####
################################################
GB_cqn_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_noiseq_filtered_cqn_normalized_samescale.txt', sep = '\t', header = TRUE, stringsAsFactors = FALSE)
GB_noiseq_filtered <- read.csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_filtered_noiseq_wt_GC', sep = '\t', header = TRUE, stringsAsFactors = FALSE, row.names = 'X')
cqn_res <- cqn(counts = GB_cqn_filtered, x = GB_noiseq_filtered$GC, lengths = GB_noiseq_filtered$total_exon_length)

factor <- colnames(counts)
columns <- factor
condition <- c('GB117', 'GB119', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123')
for (i in condition) {factor[grep((i), factor)] <- i}
coldata <- data.frame(factor, columns, row.names = 'columns')

#### CREATING PLOT ####
jpeg(file = "cqn/GB_gc_plot_after.jpeg")
cqnplot(cqn_res, n = 1, xlab = 'GC content', ylim = c(3,10))
dev.off()

jpeg(file = "cqn/GB_length_plot_after.jpeg")
cqnplot(cqn_res, n = 2, xlim = c(-2,4.5), xlab = 'length', ylim = c(4,8))
dev.off()

## DESEQ2 - DIFFERENTIAL EXPRESSION ANALYSIS

In [ ]:
#### USING DESEQ #####
condition <- c('GB117', 'GB119', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123')

dds <- DESeq(GB_dds1, parallel = TRUE)
res_GB119 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB119', 'GB117'), alpha = 0.05)
res_GB117 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB117', 'GB119'), alpha = 0.05)
res_GB49 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB49', 'GB119'), alpha = 0.05)
res_GB141 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB141', 'GB119'), alpha = 0.05)
res_GB66 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB66', 'GB119'), alpha = 0.05)
res_GB57 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB57', 'GB119'), alpha = 0.05)
res_GB13 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB13', 'GB119'), alpha = 0.05)
res_GB123 <- results(dds, pAdjustMethod = "BH", contrast = c('factor', 'GB123', 'GB119'), alpha = 0.05)

#### SUMMARY OF PROFILE ####
resOrdered_L1 <- res_GB119[order(res_GB119$pvalue),]
resOrdered_L2 <- res_GB117[order(res_GB117$pvalue),]
resOrdered_L3 <- res_GB49[order(res_GB49$pvalue),]
resOrdered_L4 <- res_GB141[order(res_GB141$pvalue),]
resOrdered_L5 <- res_GB66[order(res_GB66$pvalue),]
resOrdered_L6 <- res_GB57[order(res_GB57$pvalue),]
resOrdered_L7 <- res_GB13[order(res_GB13$pvalue),]
resOrdered_L8 <- res_GB123[order(res_GB123$pvalue),]

# summary(resOrdered_L1)
# summary(resOrdered_L2)
# summary(resOrdered_L3)
# summary(resOrdered_L4)
# summary(resOrdered_L5)
# summary(resOrdered_L6)
# summary(resOrdered_L7)
# summary(resOrdered_L8)

# #### LFC SHRINKAGE ####
# resLFC_L1 <- lfcShrink(dds, coef="factor_GB119_vs_GB119", type="apeglm")
# resLFC_L2 <- lfcShrink(dds, coef="factor_GB117_vs_GB119", type="apeglm")
# resLFC_L3 <- lfcShrink(dds, coef="factor_GB49_vs_GB119", type="apeglm")
# resLFC_L4 <- lfcShrink(dds, coef="factor_GB141_vs_GB119", type="apeglm")
# resLFC_L5 <- lfcShrink(dds, coef="factor_GB66_vs_GB119", type="apeglm")
# resLFC_L6 <- lfcShrink(dds, coef="factor_GB57_vs_GB119", type="apeglm")
# resLFC_L7 <- lfcShrink(dds, coef="factor_GB13_vs_GB119", type="apeglm")
# resLFC_L8 <- lfcShrink(dds, coef="factor_GB123_vs_GB119", type="apeglm")

#### FILES OF DIFFERNTIALLY EXPRESSED GENES ####
resSig_L1 = resOrdered_L1[which(resOrdered_L1$padj < 0.05),]
resSig_L2 = resOrdered_L2[which(resOrdered_L2$padj < 0.05),]
resSig_L3 = resOrdered_L3[which(resOrdered_L3$padj < 0.05),]
resSig_L4 = resOrdered_L4[which(resOrdered_L4$padj < 0.05),]
resSig_L5 = resOrdered_L5[which(resOrdered_L5$padj < 0.05),]
resSig_L6 = resOrdered_L6[which(resOrdered_L6$padj < 0.05),]
resSig_L7 = resOrdered_L7[which(resOrdered_L7$padj < 0.05),]
resSig_L8 = resOrdered_L8[which(resOrdered_L8$padj < 0.05),]

summary(resSig_L1)
summary(resSig_L2)
summary(resSig_L3)
summary(resSig_L4)
summary(resSig_L5)
summary(resSig_L6)
summary(resSig_L7)
summary(resSig_L8)

# #### PLOTS ####
# plotMA(res_L1, ylim=c(-5,5))
# plotMA(resLFC_L1, ylim=c(-5,5))
# plotMA(res_L2, ylim=c(-5,5))
# plotMA(resLFC_L2, ylim=c(-5,5))
# plotMA(res_L3, ylim=c(-5,5))
# plotMA(resLFC_L3, ylim=c(-5,5))
# plotMA(res_L4, ylim=c(-5,5))
# plotMA(resLFC_L4, ylim=c(-5,5))
# plotMA(res_L5, ylim=c(-5,5))
# plotMA(resLFC_L5, ylim=c(-5,5))
# plotMA(res_L6, ylim=c(-5,5))
# plotMA(resLFC_L6, ylim=c(-5,5))
# plotMA(res_L7, ylim=c(-5,5))
# plotMA(resLFC_L7, ylim=c(-5,5))
# plotMA(res_L8, ylim=c(-5,5))
# plotMA(resLFC_L8, ylim=c(-5,5))

# #### EXPORTING FILES OF DIFFERNTIALLY EXPRESSED GENES ####
# write.csv(as.data.frame(resSig_L1), file="CC2344.resSig_L1.genes")
# write.csv(as.data.frame(resSig_L2), file="CC2344.resSig_L2.genes")
# write.csv(as.data.frame(resSig_L3), file="CC2344.resSig_L3.genes")
# write.csv(as.data.frame(resSig_L4), file="CC2344.resSig_L4.genes")
# write.csv(as.data.frame(resSig_L5), file="CC2344.resSig_L5.genes")
# write.csv(as.data.frame(resSig_L6), file="CC2344.resSig_L6.genes")
# write.csv(as.data.frame(resSig_L7), file="CC2344.resSig_L7.genes")
# write.csv(as.data.frame(resSig_L8), file="CC2344.resSig_L8.genes")

#### EXPORTING LOG2FOLD CHANGE RESULTS ####
write.table(as.data.frame(res_GB119), 'genes_log2fold/GB119',  sep="\t", quote = F)
write.table(as.data.frame(res_GB117), 'genes_log2fold/GB117',  sep="\t", quote = F)
write.table(as.data.frame(res_GB49), 'genes_log2fold/GB49',  sep="\t", quote = F)
write.table(as.data.frame(res_GB141), 'genes_log2fold/GB141',  sep="\t", quote = F)
write.table(as.data.frame(res_GB66), 'genes_log2fold/GB66',  sep="\t", quote = F)
write.table(as.data.frame(res_GB57), 'genes_log2fold/GB57',  sep="\t", quote = F)
write.table(as.data.frame(res_GB13), 'genes_log2fold/GB13',  sep="\t", quote = F)
write.table(as.data.frame(res_GB123), 'genes_log2fold/GB123',  sep="\t", quote = F)

# # #PCA PLOT
# # vsdata <- vst(dds, blind=FALSE)
# # plotPCA(vsdata, intgroup="condition")

## GB STRAINS - EXPRESSION VARIATION PER INCREASING SAMPLE SIZE 

In [ ]:
GB = ['GB117', 'GB119', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123']
GB_log2Fold = pd.DataFrame()

#### COMBINING LOG2FOLD CHANGES ACROSS GENOTYPES ####
for i in GB:
    GB_log2Fold[i] = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/genes_log2fold/' + i, delimiter = "\t")['log2FoldChange']
    
#### EXPORTING LOG2FOLD VALUES ####
GB_log2Fold.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_log2Fold', sep = '\t', index = True, header = True)

GB_log2Fold = pd.read_csv('/research/projects/chlamydomonas/MAexpression/analysis/raw_counts/GB_log2Fold', delimiter = '\t', index_col = 'Unnamed: 0')
GB = ['GB117', 'GB49', 'GB141', 'GB66', 'GB57', 'GB13', 'GB123'] ## Everything relative to GB119

permutations_object = itertools.permutations(GB)
permutations_list = list(permutations_object)
GB_var= pd.DataFrame()

for a in range(len(permutations_list)):
    for i in range(7):
        df = GB_log2Fold[list(permutations_list[a][:i + 1])]
        extended_df = pd.melt(df, value_vars = list(permutations_list[a][:i + 1]))
        GB_var.at[a, str(i + 1)] = extended_df['value'].var() 

mean_expression_variance = pd.DataFrame(GB_var.mean(axis = 0), columns = ['Mean Expression Variance']).reset_index().rename(columns = {'index':'# of samples'})

sns.lineplot(data = mean_expression_variance, x = '# of samples', y = 'Mean Expression Variance')